## Recommendations for Microsoft's New Movie Studio

####  In this notebook we are going to analyze the relationship between the movie success and movie time of release,movie runtime, movie genre, directors, and actors/actresses. Success will be based on ROI and Profit margin.

#### First objective is to clean up the movie budget dataframe, then merge all the relevant data sets.

In [24]:

#importing packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from matplotlib.pyplot import figure
import sqlite3 





In [25]:
conn = sqlite3.connect('./im.db/im.db')

In [26]:
df_sql = pd.read_sql("""SELECT * FROM sqlite_master """, conn)
df_sql

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [27]:

actor = pd.read_csv('imdb.name.basics.csv.gz')
run_time = pd.read_csv('imdb.title.basics.csv.gz')
funds = pd.read_csv('tn.movie_budgets.csv.gz') 



In [28]:
run_time.head(5)

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [45]:
#copy df to work on, do not overwrite orignal 
numbers = funds.copy()
star = actor.copy()
length = run_time.copy()
player = w_d.copy()


#funtion for cleaning
def clean_a_money_column(df, series_title):
    df[series_title] = df[series_title].str.replace('$','')
    df[series_title] = df[series_title].str.replace(',','')
    df[series_title] = pd.to_numeric(df[series_title])
    return df



#remove $ string
numbers = clean_a_money_column(numbers,'worldwide_gross')
numbers = clean_a_money_column(numbers,'domestic_gross')
numbers = clean_a_money_column(numbers,'production_budget')



# new column foreign_gross, Profit Margin, ROI
numbers['net_income'] = numbers['worldwide_gross'] - numbers['production_budget']
numbers['foreign_gross'] = numbers['worldwide_gross'] - numbers['domestic_gross']
numbers['Profit Margin'] = (numbers['net_income']*100)/numbers['worldwide_gross']
numbers.loc[numbers['Profit Margin'] == float('-inf'), 'Profit Margin'] = 0
numbers['ROI'] = (numbers['net_income']) / numbers['production_budget'] * 100



# new columns month and year 
numbers['Year'] = pd.to_numeric(numbers['release_date'].map(lambda x: x[-4:]))
numbers['Month'] = numbers['release_date'].map(lambda x: x[:3])



#check for dupes
movie_dupes = pd.Series(numbers['movie'].value_counts())
title_dupes = [movie_dupes.index[index] for index in list(range(len(movie_dupes))) if movie_dupes.values[index] > 1]
numbers[numbers['movie'].isin(title_dupes)]

numbers.head(3)


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,net_income,foreign_gross,Profit Margin,ROI,Year,Month
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2351345279,2015837654,84.692106,553.257713,2009,Dec
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,804600000,60.733080,154.667286,2011,May
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,-200237650,107000000,-133.703598,-57.210757,2019,Jun


In [55]:
movies_2000 = numbers.loc[numbers['Year'] >= 2000]  
movies_2000

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,net_income,foreign_gross,Profit Margin,ROI,Year,Month
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2351345279,2015837654,84.692106,553.257713,2009,Dec
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,804600000,60.733080,154.667286,2011,May
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,-200237650,107000000,-133.703598,-57.210757,2019,Jun
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,1072413963,944008095,76.436443,324.384139,2015,May
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,999721747,696540365,75.925058,315.369636,2017,Dec
...,...,...,...,...,...,...,...,...,...,...,...,...
5776,77,"Dec 31, 2004",The Mongol King,7000,900,900,-6100,0,-677.777778,-87.142857,2004,Dec
5777,78,"Dec 31, 2018",Red 11,7000,0,0,-7000,0,0.000000,-100.000000,2018,Dec
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,1338,-3662,0,-273.692078,-73.240000,2005,Jul
5780,81,"Sep 29, 2015",A Plague So Pleasant,1400,0,0,-1400,0,0.000000,-100.000000,2015,Sep


In [58]:
movie_dupes = pd.Series(movies_2000['movie'].value_counts())
movie_dupes

Home                     3
Serenity                 2
Unknown                  2
Trance                   2
The Gift                 2
                        ..
Beauty and the Beast     1
Call of the Wild         1
Woman Thou Art Loosed    1
Yi ge ren de wu lin      1
The Edge of Seventeen    1
Name: movie, Length: 4364, dtype: int64

In [ ]:
numbers_

In [71]:
movies_2000 = movies_2000.loc['worldwide_gross'] >= 10000000

KeyError: 'worldwide_gross'

In [68]:
king_kong = numbers.loc[movies_2000['movie'] == 'King Kong']  
king_kong


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [60]:
title_dupes = [movie_dupes.index[index] for index in list(range(len(movie_dupes))) if movie_dupes.values[index] > 1]
title_dupes[0:5]

['Home', 'Serenity', 'Unknown', 'Trance', 'The Gift']

In [61]:
numbers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5782 non-null   int64  
 1   release_date       5782 non-null   object 
 2   movie              5782 non-null   object 
 3   production_budget  5782 non-null   int64  
 4   domestic_gross     5782 non-null   int64  
 5   worldwide_gross    5782 non-null   int64  
 6   net_income         5782 non-null   int64  
 7   foreign_gross      5782 non-null   int64  
 8   Profit Margin      5782 non-null   float64
 9   ROI                5782 non-null   float64
 10  Year               5782 non-null   int64  
 11  Month              5782 non-null   object 
dtypes: float64(2), int64(7), object(3)
memory usage: 542.2+ KB


In [62]:
numbers.shape

(5782, 12)

In [64]:
movies_2000[movies_2000['movie'].isin(title_dupes)]


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,net_income,foreign_gross,Profit Margin,ROI,Year,Month
38,39,"May 14, 2010",Robin Hood,210000000,105487148,322459006,112459006,216971858,34.875443,53.551908,2010,May
243,44,"Mar 27, 2015",Home,130000000,177397510,385997896,255997896,208600386,66.321060,196.921458,2015,Mar
297,98,"Aug 7, 2015",Fantastic Four,120000000,56117548,167849187,47849187,111731639,28.507250,39.874322,2015,Aug
302,3,"Oct 5, 2018",Venom,116000000,213511408,853628605,737628605,640117197,86.410952,635.886728,2018,Oct
408,9,"Nov 21, 2018",Robin Hood,99000000,30824628,84747441,-14252559,53922813,-16.817687,-14.396524,2018,Nov
484,85,"Jul 8, 2005",Fantastic Four,87500000,154696080,333132750,245632750,178436670,73.734195,280.723143,2005,Jul
883,84,"Apr 2, 2004",Hellboy,60000000,59623958,99823958,39823958,40200000,39.894189,66.373263,2004,Apr
1055,56,"Jun 16, 2000",Shaft,53012938,70327868,107190108,54177170,36862240,50.543069,102.196128,2000,Jun
1176,77,"Apr 12, 2019",Hellboy,50000000,21903748,40725492,-9274508,18821744,-22.773225,-18.549016,2019,Apr
1420,21,"Feb 18, 2011",Unknown,40000000,63686397,136123083,96123083,72436686,70.614830,240.307708,2011,Feb


In [66]:
movies_2000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4387 entries, 0 to 5781
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 4387 non-null   int64  
 1   release_date       4387 non-null   object 
 2   movie              4387 non-null   object 
 3   production_budget  4387 non-null   int64  
 4   domestic_gross     4387 non-null   int64  
 5   worldwide_gross    4387 non-null   int64  
 6   net_income         4387 non-null   int64  
 7   foreign_gross      4387 non-null   int64  
 8   Profit Margin      4387 non-null   float64
 9   ROI                4387 non-null   float64
 10  Year               4387 non-null   int64  
 11  Month              4387 non-null   object 
dtypes: float64(2), int64(7), object(3)
memory usage: 605.6+ KB


In [37]:
numbers.loc[numbers['Profit Margin'] == float('-inf'), 'Profit Margin'] = 0
numbers

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross,Profit Margin,ROI,Year,Month
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2015837654,84.692106,6.532577,2009,Dec
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,804600000,60.733080,2.546673,2011,May
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,107000000,-133.703598,0.427892,2019,Jun
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,944008095,76.436443,4.243841,2015,May
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,696540365,75.925058,4.153696,2017,Dec
...,...,...,...,...,...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,7000,0,0,0,0.000000,0.000000,2018,Dec
5778,79,"Apr 2, 1999",Following,6000,48482,240495,192013,97.505146,40.082500,1999,Apr
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,1338,0,-273.692078,0.267600,2005,Jul
5780,81,"Sep 29, 2015",A Plague So Pleasant,1400,0,0,0,0.000000,0.000000,2015,Sep


# Reading in IMDB's title.basics dataset

In [9]:
numbers.head(5)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross,Profit Margin,ROI,Year,Month
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2015837654,84.692106,6.532577,2009,Dec
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,804600000,60.733080,2.546673,2011,May
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,107000000,-133.703598,0.427892,2019,Jun
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,944008095,76.436443,4.243841,2015,May
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,696540365,75.925058,4.153696,2017,Dec


In [10]:
number_5['']

NameError: name 'number_5' is not defined

In [ ]:
length.head()

In [11]:
length.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [12]:
length['genres'].value_counts()

Documentary                   32185
Drama                         21486
Comedy                         9177
Horror                         4372
Comedy,Drama                   3519
                              ...  
Short,Thriller                    1
Comedy,Sci-Fi,Sport               1
Comedy,Fantasy,Sport              1
Crime,Musical                     1
Animation,Fantasy,Thriller        1
Name: genres, Length: 1085, dtype: int64

In [13]:
length.dropna(subset=['genres'], axis=0, inplace=True)

In [14]:
length['genres'].map(lambda x: x.split(','))

0            [Action, Crime, Drama]
1                [Biography, Drama]
2                           [Drama]
3                   [Comedy, Drama]
4          [Comedy, Drama, Fantasy]
                    ...            
146138    [Adventure, History, War]
146139                      [Drama]
146140                [Documentary]
146141                     [Comedy]
146143                [Documentary]
Name: genres, Length: 140736, dtype: object

# Genre Data Exploration 

In [15]:
length['genres'].value_counts

<bound method IndexOpsMixin.value_counts of 0            Action,Crime,Drama
1               Biography,Drama
2                         Drama
3                  Comedy,Drama
4          Comedy,Drama,Fantasy
                  ...          
146138    Adventure,History,War
146139                    Drama
146140              Documentary
146141                   Comedy
146143              Documentary
Name: genres, Length: 140736, dtype: object>